In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon

In [ ]:
gdf = gpd.read_file('tabuk-blocks-proj_sel_A.gpkg')
gdf.crs

In [ ]:
gdf.plot()

In [ ]:
gdf.head()

In [ ]:
gdf = gdf[['geometry']].copy()

In [ ]:
gdf.head()

In [ ]:
A = gdf.geometry.area.values
P = gdf.geometry.length.values
pi = np.pi

In [ ]:
# convex hull & min rotated rectangle (Shapely 2.x names)
hulls = gdf.geometry.convex_hull
mrr   = gdf.geometry.minimum_rotated_rectangle()

In [ ]:
A_hull = hulls.area.values
P_hull = hulls.length.values
A_mrr  = mrr.area.values

In [ ]:
# derive MBR sides to get aspect
def mrr_sides(poly: Polygon):
    # MRR is a 5-vertex closed ring; get unique 4 edges
    xs, ys = poly.exterior.coords.xy
    coords = list(zip(xs, ys))[:-1]
    edges = [np.hypot(coords[(i+1)%4][0]-coords[i][0],
                      coords[(i+1)%4][1]-coords[i][1]) for i in range(4)]
    # opposite sides equal; take the two unique lengths
    uniq = sorted(set(np.round(edges, 8)))
    if len(uniq) == 1:  # square
        a, b = uniq[0], uniq[0]
    else:
        a, b = uniq[-1], uniq[0]
    return a, b

sides = np.array([mrr_sides(poly) for poly in mrr])
long_side = sides.max(axis=1)
short_side = sides.min(axis=1)
aspect = long_side / np.maximum(short_side, 1e-9)

# compactness family
pp        = (4*pi*A) / (P**2)                 # Polsby–Popper
schwarz   = (2*np.sqrt(pi*A)) / P             # Schwartzberg
convexity = A / np.maximum(A_hull, 1e-9)
rect      = A / np.maximum(A_mrr, 1e-9)
rough     = P / np.maximum(P_hull, 1e-9)
shapeidx  = P / (2*np.sqrt(pi*A))

gdf["area_m2"]    = A
gdf["perimeter"]  = P
gdf["pp"]         = pp
gdf["schwarz"]    = schwarz
gdf["convexity"]  = convexity
gdf["rect"]       = rect
gdf["aspect"]     = aspect
gdf["rough"]      = rough
gdf["shape_idx"]  = shapeidx

In [ ]:
gdf

In [ ]:
gdf.describe()

In [ ]:
gdf.to_file('tabuk_metrics.gpkg')